In [1]:
#%%writefile get_searchBooks.py
# -*- coding: utf-8 -*-
#________________________________________________
import os
import django
from django.utils import timezone
#from django.utils.dateparse import parse_datetime
from datetime import datetime,date#,timezone
import pytz
from bs4 import BeautifulSoup
from bs4.element import NavigableString
from difflib import SequenceMatcher
from IPython.display import clear_output, display
from time import sleep, time
from threading import Thread
from concurrent.futures import ThreadPoolExecutor,ProcessPoolExecutor 

from fake_useragent import UserAgent
from fake_headers import Headers
from pyquery import PyQuery as pq

#
import requests
import pandas as pd
import numpy as np
import random
import re
import json
import csv
#
from get_proxy import get_proxy

#_________________________________________________
def get_searchBooks(kw:str='村上春樹',which='free',now=False):
    #kw="動盪"
    url_searchbooks="https://search.books.com.tw/search/query/cat/BKA/key/"+kw
    #
    fake_header = Headers(
        browser="chrome",  # Generate only Chrome UA
        os="win",  # Generate ony Windows platform
        headers=True  # generate misc headers
    )    
    UA=fake_header.generate()
    ippo=get_proxy(which,now)
    proxies={"http": "http://"+ippo}
    #
    r = requests.get(url_searchbooks, 
                     headers=UA,
                     proxies=proxies,
                     timeout=30)
    r.encoding='utf8'
    doc=pq(r.text)
    r.close()
    #
    searchbooks=doc.find("#searchlist .searchbook")
    items=searchbooks.find(".item")
    n=items.size()
    results=[]    
    if n==0:
        return json.dumps(results,default=str,ensure_ascii=False)
    if n>10:
        n=10
    #最多取10筆結果________________
    for i in range(n):
        book={}
        item=items.eq(i)
        #整理資料====================================
        #book['bookid']=item.find("div.input_buy input").attr('value')     
        #有些沒checkbox
        href=item.find("a[rel=mid_name]").attr('href')                
        bookid=re.search('/mid/item/(.+?)/page',href).group(1)
        book['bookid']=bookid        
        src=item.find("img.itemcov").attr("data-original")
        if 'restricted18' in src:
            #18禁先不搜
            continue
        #    
        book['src']=src
        
        book['title']=item.find("a[rel=mid_name]").text()
        #
        authors=''
        for a in item.find("a[rel=go_author]"):
            authors+=pq(a).text()+"/"
        book['author']=authors.strip("/")
        #
        book['publisher']=item.find("a[rel=mid_publish]").text()
        book['pub_dt']=re.search('出版日期[^0-9]+?([0-9\-]+)',item.text()).group(1)
        count_off= item.find('span.price').find("b").eq(0).text().ljust(2,'0')
        price_sale=item.find('span.price').find("b").eq(1).text()
        if price_sale:
            price_list=int(price_sale)*100//int(count_off)
        else:
            price_list=int(count_off) #沒有折扣
        #book['sale']=count_off+"_"+price_sale
        book['price_list']=price_list
        intro=item.find('span.price').next().text()
        intro=re.sub('\.+? *more','',intro)
        book['intro']=intro
        #
        results.append(book)    
    
    #___________________
    results=json.dumps(results,default=str,ensure_ascii=False) 
    return results

In [4]:
kw='V.S'
#%time get_searchBooks(kw)
%time get_searchBooks(kw,'OK',True)

CPU times: user 61.4 ms, sys: 5.68 ms, total: 67 ms
Wall time: 1.88 s


'[{"bookid": "F016393358", "src": "https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/F01/639/33/F016393358.jpg&w=85&h=120&v=5e620a3c", "title": "Heart of Reality: Essays on Beauty, Love, and Ethics by V. S. Soloviev", "author": "Soloviev/Vladimir Sergeyevich/Vladimir/Wozniuk", "publisher": "Ingram", "pub_dt": "2020-09-15", "price_list": 1750, "intro": ""}, {"bookid": "0010810240", "src": "https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/081/02/0010810240.jpg&w=85&h=120&v=5c24a9aa", "title": "貓頭鷹在黃昏飛翔 川上未映子V.S村上春樹訪談集", "author": "川上未映子/村上春樹", "publisher": "時報出版", "pub_dt": "2019-01-15", "price_list": 380, "intro": "作家間的頂尖對決 村上春樹V.S 川上未映子 《刺殺騎士團長》的誕生祕密、《身為職業小說家》的創作奧義、自身寫作歷程與生命的回顧、對於作品中女性所扮演的角色疑問、關於死亡與名聲的事。 那些你想知道的與你不可不知道的村上春樹"}, {"bookid": "0010636538", "src": "https://im1.book.com.tw/image/getImage?i=https://www.books.com.tw/img/001/063/65/0010636538.jpg&w=85&h=120&v=5e4130d4", "title": "民法含民法概要精修(申論題型v.s選擇題型)(增修版)", "author": "蔡裕樹", "publish

results=[]
for i in range(10):
    book={}
    item=items.eq(i)
    #
    book['bookid']=item.find("div.input_buy input").attr('value')
    book['src']=item.find("img.itemcov").attr("data-original")
    book['title']=item.find("a[rel=mid_name]").text()
    #
    authors=''
    for a in item.find("a[rel=go_author]"):
        authors+=pq(a).text()+"/"
    book['author']=authors.strip("/")
    #
    book['publisher']=item.find("a[rel=mid_publish]").text()
    book['pub_dt']=re.search('出版日期[^0-9]+?([0-9\-]+)',item.text()).group(1)
    count_off= item.find('span.price').find("b").eq(0).text().ljust(2,'0')
    price_sale=item.find('span.price').find("b").eq(1).text()
    price_list=int(price_sale)*100//int(count_off)
    #book['sale']=count_off+"_"+price_sale
    book['price_list']=price_list
    #
    results.append(book)
results 
json.dumps(results,default=str,ensure_ascii=False) 

In [2]:
'22dd'.ljust(2,'0')

'22dd'